# Import modules 

In [1]:
import pandas as pd
import qgrid
import easygui
import glob

# Helper functions

In [2]:
def count_digits(input):
    return int(math.log10(input))+1


# Import data

In [36]:
# import data

default_dir = "/Users/tanwaarpornthip/src/teaching/cpr/2020-2"
print("Select a file inside a folder that contains the raw grade file.")
target_dir = easygui.diropenbox("Select a folder that containts the raw check-in files", 
  "Select a folder that containts the raw check-in files",default=default_dir)
file_data = []
for file in glob.glob(target_dir+"/Log*.txt"):
    df = pd.read_csv(file,skiprows=1,sep='\t')
    file_data.append(df)

replacement_card_data = pd.read_csv('replacement.txt',sep='\t', header=None)

Select a file inside a folder that contains the raw grade file.


In [37]:
data = pd.concat(file_data)
data = data.drop(["No","Mchn","Name", "Unnamed: 5", "IOMd","DateTime","Unnamed: 9"],axis=1)
data = data.rename(columns={"Unnamed: 3": "Name", "EnNo":"ID"})

In [33]:
data[data['ID']<100]

,ID,Name,Mode
0,7,7,2000/01/01 00:25:19
1,7,7,2000/01/01 00:27:52
12,2,2,2021/03/13 08:49:41
64,3,3,2021/03/13 08:58:15
244,2,2,2021/03/13 14:49:29
...,...,...,...
654,5,5,2021/03/20 14:26:36
753,8,a,2021/03/20 15:03:48
778,8,a,2021/03/20 15:40:34
811,5,5,2021/03/21 16:21:23


In [38]:
replacement_card_data.to_dict()

{0: {0: 2, 1: 3, 2: 4, 3: 5, 4: 6, 5: 8},
 1: {0: 6330112036,
  1: 6330113006,
  2: 6330112049,
  3: 6330112082,
  4: 6230211007,
  5: 6330311042}}